In [28]:
import cv2
import pytesseract
import numpy as np
import glob
import os
import pandas as pd

In [29]:
def load_model( weight_path: str, cfg_path: str):
        model = cv2.dnn.readNet(weight_path, cfg_path)
        with open("obj.names", "r") as f:
            classes = [line.strip() for line in f.readlines()]
        layers_names = model.getLayerNames()
        output_layers = [layers_names[i[0]-1] for i in model.getUnconnectedOutLayers()]
        return model, layers_names, output_layers, classes

In [30]:
detector, layers_names, output_layers, classes = load_model("./weights/detection-obj_last.weights", "./weights/detection_weights/yolov4-obj.cfg")

In [31]:
def load_img(img_path):
    img = cv2.imread(img_path)
    if type(img) != None:
        height, width, channels = img.shape
        return img, height, width, channels

In [32]:
def detect_plates(img):
        blob = cv2.dnn.blobFromImage(img, scalefactor=0.00392, size=(320, 320), mean=(0, 0, 0), swapRB=True, crop=False)
        detector.setInput(blob)
        outputs = detector.forward(output_layers)
        return blob, outputs

In [33]:
def get_boxes(outputs, width, height, threshold=0.3):
        boxes = []
        confidences = []
        class_ids = []
        for output in outputs:
            for detection in output:
                scores = detection[5:]
                class_id = np.argmax(scores)
                confidence = scores[class_id]
                if confidence > threshold:
                    center_x = int(detection[0] * width)
                    center_y = int(detection[1] * height)
                    w = int(detection[2] * width)
                    h = int(detection[3] * height)
                    x = int(center_x - w / 2)
                    y = int(center_y - h / 2)
                    boxes.append([x, y, w, h])
                    confidences.append(float(confidence))
                    class_ids.append(class_id)      
        return boxes, confidences, class_ids

In [34]:
def draw_labels(boxes, confidences, class_ids, img, classes):
        indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.1, 0.1)
        font = cv2.FONT_HERSHEY_PLAIN
        plats = []
        for i in range(len(boxes)):
            if i in indexes:
                x, y, w, h = boxes[i]
                label = str(classes[class_ids[i]])
                color_green = (0, 255, 0)
                crop_img = img[y:y+h, x:x+w]
                try:
                    crop_resized = cv2.resize(crop_img, dsize=(470, 110))
                    plats.append(crop_resized)
                    cv2.rectangle(img, (x, y), (x + w, y + h), color_green, 8)
                    confidence = round(confidences[i], 3) * 100
                    cv2.putText(img, str(confidence) + "%", (x + 20, y - 20), font, 12, (0, 255, 0), 6)
                except cv2.error as err:
                    print(err)
                    
                    
        return img,plats

In [35]:
def load_model_2(weight_path: str, cfg_path: str):
        model = cv2.dnn.readNet(weight_path, cfg_path)
        with open("ocr.names", "r") as f:
            classes = [line.strip() for line in f.readlines()]
        layers_names = model.getLayerNames()
        output_layers = [layers_names[i[0]-1] for i in model.getUnconnectedOutLayers()]
        return model, layers_names, output_layers,classes
        

In [36]:
reader, layers_names_2, output_layers_2, classes_2 = load_model_2("./weights/ocr_final.weights", "./weights/ocr_weights/yolov4-ocr.cfg")

In [37]:
classes_2

['0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 'a',
 'b',
 'waw',
 'd',
 'h',
 'w',
 'ch']

In [38]:
classes

['license_plate']

In [39]:
def read_plate(img):
        blob = cv2.dnn.blobFromImage(img, scalefactor=0.00392, size=(320, 320), mean=(0, 0, 0), swapRB=True, crop=False)
        reader.setInput(blob)
        outputs = reader.forward(output_layers_2)
        return blob, outputs

In [40]:
def draw_characters(boxes, confidences, class_ids, img, classes): 
        indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.1, 0.1)
        font = cv2.FONT_HERSHEY_PLAIN
        c = 0
        characters = []
        colors = np.random.uniform(0, 255, size=(len(classes), 3))
        for i in range(len(boxes)):
            if i in indexes:
                x, y, w, h = boxes[i]
                label = str(classes[class_ids[i]])
                color = colors[i % len(colors)]
                cv2.rectangle(img, (x,y), (x+w, y+h), color, 3)
                confidence = round(confidences[i], 3) * 100
                cv2.putText(img, str(confidence) + "%", (x, y - 6), font, 1, color, 2)
                characters.append((label, x))
        characters.sort(key=lambda x:x[1])
        plate = ""
        for l in characters:
            plate += l[0]
        chg = 0

        return img, plate

In [41]:
def get_plate(img_id):
    path = 'test/'+img_id+'.jpg'
    image, height, width, channels = load_img(path)
    blob, outputs = detect_plates(image)
    boxes, confidences, class_ids = get_boxes(outputs, width, height, threshold=0.3)
    plate_img, LpImg = draw_labels(boxes, confidences, class_ids, image, classes)
    
    new_path = 'plates/'+img_id+'.jpg'
    print(img_id,len(LpImg))
    if len(LpImg):
        cv2.imwrite(new_path, LpImg[0])

In [43]:
test_df = pd.read_csv('test.csv')

In [44]:
test_df.head()

,image_id,plate_string
0,450,NaN
1,451,NaN
2,452,NaN
3,453,NaN
4,454,NaN


In [45]:
IDs = np.array(test_df.image_id)
IDs

array([450, 451, 452, 453, 454, 455, 456, 457, 458, 459, 460, 461, 462,
       463, 464, 465, 466, 467, 468, 469, 470, 471, 472, 473, 474, 475,
       476, 477, 478, 479, 480, 481, 482, 483, 484, 485, 486, 487, 488,
       489, 490, 491, 492, 493, 494, 495, 496, 497, 498, 499, 500, 501,
       502, 503, 504, 505, 506, 507, 508, 509, 510, 511, 512, 513, 514,
       515, 516, 517, 518, 519, 520, 521, 522, 523, 524, 525, 526, 527,
       528, 529, 530, 531, 532, 533, 534, 535, 536, 537, 538, 539, 540,
       541, 542, 543, 544, 545, 546, 547, 548, 549, 550, 551, 552, 553,
       554, 555, 556, 557, 558, 559, 560, 561, 562, 563, 564, 565, 566,
       567, 568, 569, 570, 571, 572, 573, 574, 575, 576, 577, 578, 579,
       580, 581, 582, 583, 584, 585, 586, 587, 588, 589, 590, 591, 592,
       593, 594, 595, 596, 597, 598, 599, 600, 601, 602, 603, 604, 605,
       606, 607, 608, 609, 610, 611, 612, 613, 614, 615, 616, 617, 618,
       619, 620, 621, 622, 623, 624, 625, 626, 627, 628, 629, 63

In [46]:
for i in range(len(IDs)):
    get_plate(str(IDs[i]))

450 1
451 1
452 1
453 1
454 1
455 1
456 1
457 1
458 1
459 1
460 1
461 1
462 1
463 1
464 1
465 1
466 1
467 1
468 2
469 2
470 1
471 1
472 1
473 1
474 1
475 1
476 1
477 1
478 1
479 1
480 1
481 1
482 1
483 1
484 1
485 1
486 1
487 0
488 1
489 1
490 1
491 1
492 1
493 1
494 1
495 1
496 1
497 0
498 1
499 0
500 1
501 1
502 1
503 1
504 1
505 1
506 1
507 1
508 1
509 1
510 1
511 1
512 1
513 1
514 1
515 1
516 1
517 1
518 1
519 1
520 1
521 1
522 0
523 1
524 1
525 1
526 1
527 1
528 1
529 1
530 1
531 1
532 1
533 1
534 1
535 1
536 1
537 1
538 1
539 1
540 1
541 1
542 1
543 1
544 1
545 1
546 1
547 1
548 2
549 2
550 1
551 1
552 1
553 1
554 1
555 1
556 1
557 1
558 1
559 1
560 1
561 1
562 1
563 1
564 1
565 1
566 1
567 1
568 1
569 1
570 1
571 1
572 1
573 1
574 1
575 2
576 1
577 1
578 1
579 1
580 1
581 1
582 1
583 1
584 1
585 1
586 1
587 1
588 1
589 1
590 1
591 1
592 1
593 1
594 1
595 1
596 1
597 1
598 1
599 1
600 1
601 1
602 1
603 1
604 1
605 1
606 1
607 1
608 1
609 1
610 1
611 1
612 1
613 1
614 1
615 1
616 

In [48]:
def get_plate(img_id):
    img_id = str(img_id)
    filename= img_id+'.jpg'
    path = 'plates/'+img_id+'.jpg'
    if os.path.exists(path):
        image, height, width, channels = load_img(path)
        blob, outputs = read_plate(image)
        boxes, confidences, class_ids = get_boxes(outputs, width, height, threshold=0.3)
        segmented, plate_text = draw_characters(boxes, confidences, class_ids, image, classes_2)
        print(path, plate_text)
        return plate_text
    return ''
    

In [126]:
os.walk('plates/')

<generator object walk at 0x0000023082468660>

In [50]:
test_pd.plate_string = test_pd['image_id'].apply(get_plate)

plates/450.jpg 68628waw6
plates/451.jpg 31838a11
plates/452.jpg 3a183811
plates/453.jpg 16038a11
plates/454.jpg 87000a1
plates/455.jpg 24710a33
plates/456.jpg 4106b6
plates/457.jpg 22837h6
plates/458.jpg 47347a11
plates/459.jpg 37145a11
plates/460.jpg 50767a11
plates/461.jpg 97132a40
plates/462.jpg 40070a11
plates/463.jpg 40070a11
plates/464.jpg 585060ww
plates/465.jpg 585060ww
plates/466.jpg 2a7
plates/467.jpg 3579a14
plates/468.jpg 19413h6
plates/469.jpg a7
plates/470.jpg 0308130
plates/471.jpg 594594ww
plates/472.jpg 24808a44
plates/473.jpg 587943ww
plates/474.jpg 587943ww
plates/475.jpg 1850a7
plates/476.jpg 60391d1
plates/477.jpg 35341a72
plates/478.jpg 2a738161
plates/479.jpg 972h6
plates/480.jpg 10013a57
plates/481.jpg 40970a48
plates/482.jpg 90225d1
plates/483.jpg 9225d1
plates/484.jpg 50616a11
plates/485.jpg 84916h6
plates/486.jpg 1511waw6
plates/488.jpg 37761a11
plates/489.jpg 45881a14
plates/490.jpg b4542968
plates/491.jpg 914waw1
plates/492.jpg 76138a72
plates/493.jpg 31766

In [51]:
test_pd.plate_string = test_pd.plate_string.str.replace('ch','p')

In [52]:
test_pd[test_pd['plate_string']==''].count()

image_id        3
plate_string    3
dtype: int64

In [53]:
test_pd.head()

,image_id,plate_string
0,450,68628waw6
1,451,31838a11
2,452,3a183811
3,453,16038a11
4,454,87000a1


In [54]:
test_pd.to_csv('submission8.csv', index=False)